In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from datasets import load_from_disk
import pandas as pd
from tqdm.notebook import tqdm
import time

In [3]:
dataset_dict = load_from_disk("../../data/complete_dataset/")

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 495
    })
    validation: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 124
    })
})


In [4]:
import pandas as pd

validation_df = pd.DataFrame(dataset_dict['validation'])
df = validation_df[['answer_urdu']].copy()
df['id'] = range(len(df))
df.rename(columns={'answer_urdu': 'answers'}, inplace=True)
df = df[['id', 'answers']]

display(df)

,id,answers
0,0,اسلامی کونسل کا کوئی رکن تین سال کی مدت کے لیے...
1,1,پاکستان میں یوم آغاز سے قبل قبائلی علاقے جات م...
2,2,نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی ک...
3,3,وفاقی حکومت یا، جیسی بھی صورت ہو، صوبائی حکومت...
4,4,کسی رکن کو منحرف قرار دینے سے پہلے، پارٹی کے س...
...,...,...
119,119,وزیر اعظم مشترکہ مفادات کی کونسل کا چیئرمین ہو...
120,120,نہیں، کسی شخص کو شہادت حاصل کرنے کی غرض سے اذی...
121,121,سینیٹ میں اسپیکر اور ڈپٹی اسپیکر کے مساوی عہدے...
122,122,مالی بل کی ابتدا قومی اسمبلی میں ہوگی۔


In [5]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [6]:
def get_urdu_response(context, question, model_name="gemini-1.5-pro-002"):
    # Create the model
    generation_config = {
        "temperature": 0.0,
        "max_output_tokens": 1024,
        "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
        model_name=model_name,
        generation_config=generation_config,
        system_instruction="You are a Pakistani legal expert who speaks in Urdu. You only respond in Urdu script."
    )

    chat_session = model.start_chat(history=[])

    prompt = f"""Below is a context provided, followed by a question. Your task is to answer the question based on the given context. The answer should consist only of the response to the question in Urdu and should not include any other information or commentary.

    Context: ```{context}```

    Question: ```{question}```

    Answer:"""

    response = chat_session.send_message(prompt)

    return response.text.strip()

In [7]:
def get_and_save_answers(model_name):
    predicted_answers = []

    for i in tqdm(range(len(dataset_dict['validation'])), desc="Processing responses", unit="item"):
        test_point = dataset_dict['validation'][i]

        context = test_point['context_urdu']
        question = test_point['question_urdu']

        tries = 0
        max_tries = 5

        predicted_answer = None

        # retry n times else break - due to quota limit error occuring sometimes
        while tries < max_tries:
            tries += 1

            try:
                predicted_answer = get_urdu_response(
                    context=context, question=question, model_name=model_name)
                break
            except Exception as error:
                error_msg = f"Index: {i} - ERROR OCCURED: {error}"
                predicted_answer = error_msg

                time.sleep(15)
        
        if tries >= max_tries:
            print(predicted_answer)

        predicted_answers.append(predicted_answer)

    for i in range(5):
        print(predicted_answers[i])

    df2 = df.copy()
    df2['prediction_text_gemini'] = predicted_answers

    display(df2.head())

    df2.to_csv(
        f'../../data/model_qa_responses/{model_name}-responses.csv', index=False)

In [8]:
models = [
    'gemini-1.5-pro-002',
    'gemini-1.5-flash-002'
]

In [9]:
for model in models:
    print(f'Model: {model}')

    get_and_save_answers(model)

    print('---' * 10, 'Done', '---' * 10)

Model: gemini-1.5-pro-002


Processing responses:   0%|          | 0/124 [00:00<?, ?item/s]

تین سال۔
بلوچستان اور خیبر پختونخوا کے قبائلی علاقہ جات، اور امب، چترال، دیر اور سوات کی سابقہ ریاستیں۔
نہیں۔ کسی جج کا تبادلہ اس کی رضامندی کے بغیر نہیں کیا جا سکتا۔
وفاقی حکومت یا صوبائی حکومت کے پاس، جیسا کہ صورتحال ہو۔
پارٹی کا سربراہ رکن کو اظہارِ وجوہ کا موقع فراہم کرے گا کہ کیوں نہ اس کے خلاف منحرف ہونے کا اعلان کر دیا جائے۔


,id,answers,prediction_text_gemini
0,0,اسلامی کونسل کا کوئی رکن تین سال کی مدت کے لیے...,تین سال۔
1,1,پاکستان میں یوم آغاز سے قبل قبائلی علاقے جات م...,بلوچستان اور خیبر پختونخوا کے قبائلی علاقہ جات...
2,2,نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی ک...,نہیں۔ کسی جج کا تبادلہ اس کی رضامندی کے بغیر ن...
3,3,وفاقی حکومت یا، جیسی بھی صورت ہو، صوبائی حکومت...,وفاقی حکومت یا صوبائی حکومت کے پاس، جیسا کہ صو...
4,4,کسی رکن کو منحرف قرار دینے سے پہلے، پارٹی کے س...,پارٹی کا سربراہ رکن کو اظہارِ وجوہ کا موقع فرا...


------------------------------ Done ------------------------------
Model: gemini-1.5-flash-002


Processing responses:   0%|          | 0/124 [00:00<?, ?item/s]

تین سال
یومِ آغاز سے قبل پاکستان میں بلوچستان اور خیبر پختونخوا کے قبائلی علاقے جات، امب، چترال، دیر اور سوات کی سابقہ ریاستیں قبائلی علاقے جات سمجھی جاتی تھیں۔
نہیں۔
وفاقی حکومت یا متعلقہ صوبائی حکومت۔
پارٹی کے سربراہ کو کسی رکن کو منحرف قرار دینے سے پہلے اسے اظہار وجوہ کا موقع فراہم کرنا چاہیے۔


,id,answers,prediction_text_gemini
0,0,اسلامی کونسل کا کوئی رکن تین سال کی مدت کے لیے...,تین سال
1,1,پاکستان میں یوم آغاز سے قبل قبائلی علاقے جات م...,یومِ آغاز سے قبل پاکستان میں بلوچستان اور خیبر...
2,2,نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی ک...,نہیں۔
3,3,وفاقی حکومت یا، جیسی بھی صورت ہو، صوبائی حکومت...,وفاقی حکومت یا متعلقہ صوبائی حکومت۔
4,4,کسی رکن کو منحرف قرار دینے سے پہلے، پارٹی کے س...,پارٹی کے سربراہ کو کسی رکن کو منحرف قرار دینے ...


------------------------------ Done ------------------------------
